In [1]:
import sys
from pathlib import Path
project_root = Path('./').resolve().parent
sys.path.append(str(project_root))

In [2]:
from utils.utils import load_yaml_config, concordance_index
from models.ensembles import Ensembles
import numpy as np
from matminer.featurizers.composition import ElementProperty
import pandas as pd

In [12]:
def wasted_time(true_val,pred_val):
    times=[]
    for i in range(len(true_val)):
        x=pred_val[i]-true_val[i]
        if(x>=0):
            times.append(x)
        else:
            times.append(pred_val[i])
    return times

In [3]:
df=pd.read_pickle('../data/cgcnn_junwen_oct_6_kdist/junwen_6_oct_no_structural_duplicates_kdist_with_supercell.pkl')

In [4]:
da=pd.read_pickle('../data/cgcnn_junwen_oct_6_kdist/wall_times.pkl')

In [5]:
for i in range(len(da['Source ID'].values)):
    da['Source ID'].values[i]=str(da['Source ID'].values[i])

# ALIGGNd

In [6]:
dflower=pd.read_csv('../output/alignn/quantile5.csv')
dfhigher=pd.read_csv('../output/alignn/quantile95.csv')
df50=pd.read_csv('../output/alignn//quantile50.csv')

from sklearn.model_selection import train_test_split

index=df50.index.values
cal_ind, test_ind = train_test_split(index, test_size=0.5, random_state=42)
cal_y_lower=dfhigher['prediction'].values[cal_ind]
test_y_lower=dfhigher['prediction'].values[test_ind]
cal_y_med=df50['prediction'].values[cal_ind]
test_y_med=df50['prediction'].values[test_ind]
cal_y_upper=dflower['prediction'].values[cal_ind]
test_y_upper=dflower['prediction'].values[test_ind]
ycal=df50['truth'].values[cal_ind]
ytest=df50['truth'].values[test_ind]

In [7]:
test_ind_original=df50['id'].values[test_ind]

In [8]:
testset=pd.DataFrame()
testset['source_db_id']=df.iloc[test_ind_original]['source_db_id'].values
testset['ytest']=ytest
testset['ypred']=test_y_med
testset['ylower']=test_y_lower
# CQR90: c_opt=0.158, CQR80: c_opt=0.184, CQR70: c_opt=0.205
c_opt=0.158
testset['ydummy']=c_opt*np.ones(len(testset))
dummy_times=[]
lower_times=[]
true_times=[]
predicted_times=[]
ypred_corr=[]

In [9]:
for i in range(len(testset)):
    ind=testset.iloc[i]['source_db_id']
    dx=da.loc[da['Source ID']==ind]
    true_k_dist=testset.iloc[i]['ytest']
    true_time=dx.loc[np.abs(dx['k_dist']-true_k_dist)<0.002]['wall_time_s'].values[0]
    true_times.append(true_time)
    try:
        predicted_k_dist=testset.iloc[i]['ypred']
        ind_pred=np.searchsorted(np.array(predicted_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        ypred_corr.append(dx.iloc[ind_pred]['k_dist'])
        predicted_time=dx.iloc[ind_pred]['wall_time_s']
    except:
        predicted_time=np.nan
        ypred_corr.append(np.nan)
    predicted_times.append(predicted_time)
    try:
        lower_k_dist=testset.iloc[i]['ylower']
        ind_lower=np.searchsorted(np.array(lower_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        lower_time=dx.iloc[ind_lower]['wall_time_s']
    except:
        lower_time=np.nan
    lower_times.append(lower_time)
    try:
        dummy_k_dist=testset.iloc[i]['ydummy']
        ind_dummy=np.searchsorted(np.array(dummy_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        dummy_time=dx.iloc[ind_dummy]['wall_time_s']
    except:
        dummy_time=np.nan
    dummy_times.append(dummy_time)

In [10]:
testset['ypred_corr']=ypred_corr
testset['true_times']=true_times
testset['predicted_times']=predicted_times
testset['lower_times']=lower_times
testset['dummy_times']=dummy_times

In [11]:
testset1=testset.loc[~testset['ypred_corr'].isna()]
dy=testset.loc[~testset['lower_times'].isna()]
dyy=dy.loc[~dy['dummy_times'].isna()]

In [13]:
cost_dummy=wasted_time(dyy['true_times'].values,dyy['dummy_times'].values)
cost_lower=wasted_time(dyy['true_times'].values,dyy['lower_times'].values)
cost_pred=wasted_time(dyy['true_times'].values,dyy['predicted_times'].values)

In [14]:
np.mean(cost_dummy),np.mean(cost_lower)

(np.float64(469.35808405438814), np.float64(370.2410630407911))

In [15]:
np.mean(dyy['true_times'])

np.float64(355.22625463535235)

In [16]:
np.mean(cost_dummy)-np.mean(cost_lower), (np.mean(cost_dummy)-np.mean(cost_lower))/np.mean(cost_dummy)

(np.float64(99.11702101359703), np.float64(0.21117569800312969))

# RF+CSLM

In [17]:
path_config = Path('./').resolve().parent / 'configs' / 'ensembles.yaml'
config = load_yaml_config(path_config)
config['data']['root_dir']=Path('./').resolve().parent / config['data']['root_dir']
config['features']['checkpoint_path']=Path('./').resolve().parent / config['features']['checkpoint_path']
model = Ensembles(**config)
model.prep_data()

RandomForestRegressor(random_state=11)


/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/io/cif.py:1324: UserWarning: Issues encountered while parsing CIF: 16 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/io/cif.py:1324: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/io/cif.py:1324: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/io/cif.py:1

** Features are being calculated **


/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/libpyg.so, 0x0006): Library not loaded: /Library/Frameworks/Python.framework/Versions/3.11/Python
  Referenced from: <CA14ED34-FA3D-31FE-B4AD-2B2A8446B324> /opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/libpyg.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.11/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.11/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.11/Python' (no such file)
  import torch_geometric.typing
/opt/miniconda3/envs/kpoints/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/opt/miniconda3/envs/kpoint

test_ratio 1.0, train_ratio 0.0
calibration False, cal_ratio 0.0


In [18]:
from sklearn.model_selection import train_test_split
train_ind, test_ind = train_test_split(model.data.index.values, 
                                               test_size=0.2, 
                                               random_state=42)
cal_ind, test_ind = train_test_split(test_ind, 
                                               test_size=0.5, 
                                               random_state=42)
ytrain = np.array(model.data.iloc[train_ind][1].values,dtype='float')
ytest = np.array(model.data.iloc[test_ind][1].values,dtype='float')
ycal = np.array(model.data.iloc[cal_ind][1].values,dtype='float')
Xtrain = model.features[train_ind,:]
Xtest = model.features[test_ind,:]
Xcal = model.features[cal_ind,:]

In [33]:
from sklearn_quantile import RandomForestQuantileRegressor
import pickle

with open("../trained_models/RF/QRF95.pkl", "rb") as f:
    model1 = pickle.load(f)

In [34]:
predictions = model1.predict(Xtest)
cal_predictions = model1.predict(Xcal)
test_y_lower = predictions[0]
test_y_med = predictions[1]
test_y_upper = predictions[2]
cal_y_lower = cal_predictions[0]
cal_y_med = cal_predictions[1]
cal_y_upper = cal_predictions[2]

In [35]:
test_ind_original=test_ind

In [36]:
testset=pd.DataFrame()
testset['source_db_id']=df.iloc[test_ind_original]['source_db_id'].values
testset['ytest']=ytest
testset['ypred']=test_y_med
testset['ylower']=test_y_lower
# CQR90: c_opt=0.158, CQR80: c_opt=0.184, CQR70: c_opt=0.205
c_opt=0.158
testset['ydummy']=c_opt*np.ones(len(testset))
dummy_times=[]
lower_times=[]
true_times=[]
predicted_times=[]
ypred_corr=[]

In [37]:
for i in range(len(testset)):
    ind=testset.iloc[i]['source_db_id']
    dx=da.loc[da['Source ID']==ind]
    true_k_dist=testset.iloc[i]['ytest']
    true_time=dx.loc[np.abs(dx['k_dist']-true_k_dist)<0.002]['wall_time_s'].values[0]
    true_times.append(true_time)
    try:
        predicted_k_dist=testset.iloc[i]['ypred']
        ind_pred=np.searchsorted(np.array(predicted_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        ypred_corr.append(dx.iloc[ind_pred]['k_dist'])
        predicted_time=dx.iloc[ind_pred]['wall_time_s']
    except:
        predicted_time=np.nan
        ypred_corr.append(np.nan)
    predicted_times.append(predicted_time)
    try:
        lower_k_dist=testset.iloc[i]['ylower']
        ind_lower=np.searchsorted(np.array(lower_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        lower_time=dx.iloc[ind_lower]['wall_time_s']
    except:
        lower_time=np.nan
    lower_times.append(lower_time)
    try:
        dummy_k_dist=testset.iloc[i]['ydummy']
        ind_dummy=np.searchsorted(np.array(dummy_k_dist*np.ones(len(dx))-dx['k_dist']),0)
        dummy_time=dx.iloc[ind_dummy]['wall_time_s']
    except:
        dummy_time=np.nan
    dummy_times.append(dummy_time)

In [38]:
testset['ypred_corr']=ypred_corr
testset['true_times']=true_times
testset['predicted_times']=predicted_times
testset['lower_times']=lower_times
testset['dummy_times']=dummy_times

In [39]:
testset1=testset.loc[~testset['ypred_corr'].isna()]
dy=testset.loc[~testset['lower_times'].isna()]
dyy=dy.loc[~dy['dummy_times'].isna()]

In [40]:
cost_dummy=wasted_time(dyy['true_times'].values,dyy['dummy_times'].values)
cost_lower=wasted_time(dyy['true_times'].values,dyy['lower_times'].values)
cost_pred=wasted_time(dyy['true_times'].values,dyy['predicted_times'].values)

In [41]:
np.mean(cost_dummy),np.mean(cost_lower)

(np.float64(469.8790865384616), np.float64(353.23044471153844))

In [42]:
np.mean(dyy['true_times'])

np.float64(363.8950240384615)

In [43]:
np.mean(cost_dummy)-np.mean(cost_lower), (np.mean(cost_dummy)-np.mean(cost_lower))/np.mean(cost_dummy)

(np.float64(116.64864182692315), np.float64(0.2482524657274249))